In [1]:
import pandas as pd
import numpy as np
import sys
import random
from sqlalchemy import create_engine

from snorkel.labeling import labeling_function

## Load Data

### DataFrame

In [2]:
## load the data by choosing a random 10% of the data
## is the data ordered? am I missing a large chunk due to my partial loading?
# p = 0.01
# responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=lambda i: i>0 and random.random() > p)

### SQL Engine

In [3]:
# access SQL DB with data
engine = create_engine("sqlite:///../data/moralmachine.db", echo=False)

In [4]:
# demo
# responses = pd.read_sql("SELECT * FROM sharedresponses ORDER BY RANDOM() LIMIT 100000", con=engine)
# responses.columns

In [5]:
# ## For viewing the possible values for each feature
# for feature in responses.columns:
#     print(feature)
#     print(responses[feature].unique())
#     print(responses[feature].unique().shape)

In [6]:
# ## What does a single user's session/response look like?
# responses_random = responses[responses['ScenarioTypeStrict'] == 'Random']
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     ## see a random user's session
# #     display(responses_random[responses_random['ExtendedSessionID'] == responses_random['ExtendedSessionID'].sample().values[0]])
    
#     ## see the user with nth most responses
#     n = 100
#     display(responses_random[responses_random['ResponseID'] == responses_random.groupby(by='ResponseID').size().sort_values(ascending=False).index[n]]) 

In [7]:
# ## group alternatives pairwise!
# # take only responses where both alternatives are present in the random sample - for a full sample, this step isn't necessary
# responses_grouped = responses.groupby(by='ResponseID').filter(lambda g: g.shape[0] < 2)
# responses_grouped.shape

## Data Exploration

In [8]:
# responses.describe()

## Snorkel Labeling
Using https://www.snorkel.org/use-cases/01-spam-tutorial

In [9]:
# query random sample of responses grouped by ResponseID; only take responses for which both instances are present
query = """
    --SELECT * FROM (
        SELECT * FROM sharedresponses
            WHERE ScenarioTypeStrict LIKE 'Random'
        ORDER BY RANDOM()
        LIMIT 10000
    --)
    --GROUP BY ResponseID, ExtendedSessionID, UserID
    --HAVING COUNT(ResponseID) > 1
"""
df = pd.read_sql(query, con=engine).groupby(by='ResponseID').filter(lambda g: g.shape[0] > 1).sort_values('ResponseID')
df

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
1275,5JjfvpkcXn8vxTog3,-1136159268_1349835919,1349835919,10,1,0,0,1,Rand,Random,...,0,0,0,0,0,1,0,0,0,1
1403,5JjfvpkcXn8vxTog3,-1136159268_1349835919,1349835919,10,0,0,1,0,Rand,Random,...,0,1,0,0,0,0,0,0,0,0
7338,6fJ9s9FcMsRiDWkXn,1714315090_2923411722714977.0,2923411722714980,4,1,0,0,2,Rand,Random,...,0,1,0,0,0,0,0,0,0,0
6538,6fJ9s9FcMsRiDWkXn,1714315090_2923411722714977.0,2923411722714980,4,0,0,1,0,Rand,Random,...,0,0,0,0,0,0,0,0,1,0
3737,87265jbikmSyjgHhH,-725684815_2011237781010055.0,2011237781010060,11,0,0,0,2,Rand,Random,...,0,0,0,0,1,1,0,0,0,0
5084,87265jbikmSyjgHhH,-725684815_2011237781010055.0,2011237781010060,11,1,0,1,0,Rand,Random,...,0,0,0,0,1,0,0,0,0,0
1538,EhQqsjMQSq57CCEbB,-243201999_668148619,668148619,8,1,0,0,2,Rand,Random,...,0,0,0,0,0,2,0,0,0,0
6979,EhQqsjMQSq57CCEbB,-243201999_668148619,668148619,8,0,0,1,0,Rand,Random,...,0,0,0,0,0,0,0,0,0,0
1013,Groo5iP96jHc75XKe,845277464_9676384761966740.0,9676384761966740,12,0,0,1,0,More,Random,...,0,0,0,0,1,0,1,0,0,0
4430,Groo5iP96jHc75XKe,845277464_9676384761966740.0,9676384761966740,12,1,0,0,2,Less,Random,...,0,0,0,0,1,0,0,0,0,0


Need to have both alternatives in the same tuple, marking them by whether or not they are the intervention - specifically the factors:
> 'NumberOfCharacters', 'DiffNumberOfCharacters', 'Saved', 'Template', 'DescriptionShown',
'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
'Cat'

In [10]:
df[["ResponseID","Intervention","Saved"]]

,ResponseID,Intervention,Saved
1275,5JjfvpkcXn8vxTog3,1,1
1403,5JjfvpkcXn8vxTog3,0,0
7338,6fJ9s9FcMsRiDWkXn,1,0
6538,6fJ9s9FcMsRiDWkXn,0,1
3737,87265jbikmSyjgHhH,0,1
5084,87265jbikmSyjgHhH,1,0
1538,EhQqsjMQSq57CCEbB,1,0
6979,EhQqsjMQSq57CCEbB,0,1
1013,Groo5iP96jHc75XKe,0,1
4430,Groo5iP96jHc75XKe,1,0


In [19]:
scenario_fields = [
    'ResponseID', 'Barrier', 'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'DescriptionShown', 'LeftHand', 'Man', 'Woman', 'Pregnant', 'Stroller', \
    'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', \
    'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat'
]
intervention = df[df['Intervention'] == 1][scenario_fields].set_index('ResponseID')
no_intervention = df[df['Intervention'] == 0][scenario_fields].set_index('ResponseID')
df_joined = intervention.join(no_intervention, lsuffix='_int', rsuffix='_noint', how='inner')
df_joined = df[[col for col in df.columns if col not in scenario_fields or col == 'ResponseID']].set_index('ResponseID').join(df_joined)
df_joined['Intervened'] = (df_joined['Saved_int'] == 1).astype(int)
df_joined = df_joined.drop(axis='columns', labels=['Saved_{}'.format(s) for s in ['int', 'noint']])
df_joined

,Intervened,Saved_int,Saved_noint
ResponseID,,,
5JjfvpkcXn8vxTog3,1,1,0
5JjfvpkcXn8vxTog3,1,1,0
6fJ9s9FcMsRiDWkXn,0,0,1
6fJ9s9FcMsRiDWkXn,0,0,1
87265jbikmSyjgHhH,0,0,1
87265jbikmSyjgHhH,0,0,1
EhQqsjMQSq57CCEbB,0,0,1
EhQqsjMQSq57CCEbB,0,0,1
Groo5iP96jHc75XKe,0,0,1


A standard train test split for testing:

In [12]:
from sklearn.model_selection import train_test_split

X = df_joined.drop(labels=["Intervened"], axis='columns', inplace=False)
y = df_joined["Intervened"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.2, random_state=1)

Some sample labeling functions:

In [13]:
sys.path.insert(1, '../heuristics')
import labeling_functions
import utils
import importlib
importlib.reload(labeling_functions)
importlib.reload(utils)
from snorkel.labeling import PandasLFApplier

lfs = [
    labeling_functions.doctors,
    labeling_functions.utilitarian,
    labeling_functions.utilitarian_anthro,
    labeling_functions.inaction,
    labeling_functions.pedestrians,
    labeling_functions.females,
    labeling_functions.fitness,
    labeling_functions.status,
    labeling_functions.legal,
    labeling_functions.illegal,
    labeling_functions.youth,
    labeling_functions.criminals,
    labeling_functions.homeless,
    labeling_functions.pets,
    labeling_functions.spare_strollers,
    labeling_functions.spare_girl,
    labeling_functions.spare_boy,
    labeling_functions.spare_pregnant
]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=X_train)
L_dev = applier.apply(df=X_dev)
L_train

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1/1 [00:00<00:00, 306.76it/s]

ExtendedSessionID     -725684815_2011237781010055.0
UserID                             2011237781010060
ScenarioOrder                                    11
Intervention                                      0
PedPed                                            0
                                  ...              
MaleAthlete_noint                                 1
FemaleDoctor_noint                                0
MaleDoctor_noint                                  0
Dog_noint                                         0
Cat_noint                                         0
Name: 87265jbikmSyjgHhH, Length: 64, dtype: object
ExtendedSessionID     -725684815_2011237781010055.0
UserID                             2011237781010060
ScenarioOrder                                    11
Intervention                                      0
PedPed                                            0
                                  ...              
MaleAthlete_noint                                 1
FemaleDoctor_

array([[-1, -1, -1,  0,  1, -1,  0, -1, -1,  0, -1, -1, -1, -1, -1, -1,
        -1, -1],
       [ 0,  0,  0,  0,  0,  0, -1, -1, -1,  1,  0, -1, -1, -1,  1, -1,
        -1, -1],
       [-1,  0,  0,  0,  0, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1],
       [-1,  1,  1,  0,  1, -1,  1,  1, -1, -1,  0, -1, -1, -1,  1, -1,
        -1, -1],
       [-1,  0, -1,  0,  1,  0,  1, -1, -1, -1,  0, -1, -1, -1,  0,  1,
        -1,  1],
       [-1, -1, -1,  0,  1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1],
       [ 0,  0,  0,  0,  1,  0, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,
         1, -1],
       [-1,  1,  1,  0,  1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,
         0, -1],
       [-1, -1, -1,  0,  1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1],
       [-1, -1, -1,  0,  1, -1,  0, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1],
       [-1,  1,  1,  0,  1, -1, -1,  1, -1,  0,  1, -1, -1, -1, -1, -1,
         0, -1],
       [-1,  1,  1,  

In [14]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
doctors,0,"[0, 1]",0.266667,0.266667,0.266667
utilitarian,1,"[0, 1]",0.666667,0.666667,0.600000
utilitarian_anthro,2,"[0, 1]",0.666667,0.666667,0.600000
inaction,3,[0],1.000000,1.000000,0.933333
pedestrians,4,"[0, 1]",1.000000,1.000000,0.933333
females,5,[0],0.333333,0.333333,0.333333
fitness,6,"[0, 1]",0.533333,0.533333,0.533333
status,7,"[0, 1]",0.266667,0.266667,0.200000
legal,8,[1],0.066667,0.066667,0.066667
illegal,9,"[0, 1]",0.333333,0.333333,0.333333


In [15]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=y_dev.values).sort_values("Emp. Acc.", ascending=False)

ValueError: Input could not be converted to 1d np.array

In [16]:
X_train.columns

Index(['ExtendedSessionID', 'UserID', 'ScenarioOrder', 'Intervention',
       'PedPed', 'CrossingSignal', 'AttributeLevel', 'ScenarioTypeStrict',
       'ScenarioType', 'DefaultChoice', 'NonDefaultChoice',
       'DefaultChoiceIsOmission', 'Template', 'UserCountry3', 'Barrier_int',
       'NumberOfCharacters_int', 'DiffNumberOFCharacters_int',
       'DescriptionShown_int', 'LeftHand_int', 'Man_int', 'Woman_int',
       'Pregnant_int', 'Stroller_int', 'OldMan_int', 'OldWoman_int', 'Boy_int',
       'Girl_int', 'Homeless_int', 'LargeWoman_int', 'LargeMan_int',
       'Criminal_int', 'MaleExecutive_int', 'FemaleExecutive_int',
       'FemaleAthlete_int', 'MaleAthlete_int', 'FemaleDoctor_int',
       'MaleDoctor_int', 'Dog_int', 'Cat_int', 'Barrier_noint',
       'NumberOfCharacters_noint', 'DiffNumberOFCharacters_noint',
       'DescriptionShown_noint', 'LeftHand_noint', 'Man_noint', 'Woman_noint',
       'Pregnant_noint', 'Stroller_noint', 'OldMan_noint', 'OldWoman_noint',
       'Boy_n